# Evn

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
# gpu 사용량 확인
!nvidia-smi

In [ ]:
# data dir (환경에 맞게 수정하세요.)
data_dir = '/content/drive/MyDrive/문서/강의계획서/삼성전기/삼성전기.20220228/data'
os.listdir(data_dir)

# Sigmoid

In [ ]:
val_x = np.linspace(-10, 10, 1000)
val_x

In [ ]:
sigmoid_y = 1 / (1 + np.exp(-val_x))
# draw plot
plt.plot(val_x, sigmoid_y)
plt.show()

In [ ]:
dev_sigmoid_y = 1 / (1 + np.exp(-val_x)) * (1 - 1 / (1 + np.exp(-val_x)))
# draw plot
plt.plot(val_x, dev_sigmoid_y)
plt.show()

In [ ]:
dev_sigmoid_y = sigmoid_y * (1 - sigmoid_y)
# draw plot
plt.plot(val_x, dev_sigmoid_y)
plt.show()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def sigmoid_deriv(y):
    return y * (1 - y)

# Layer1

In [ ]:
x = np.array([
    [0, 0, 1],
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
])  # (bs, 3)

In [ ]:
y_true = np.array([[0, 1, 1, 0]]).T
y_true  # (bs, 1)

In [ ]:
lr = 0.01

In [ ]:
Wo = np.random.randn(3, 1)  # (3, 1)
bo = np.zeros((1,))  # (1,)

Wo, bo

## forward

In [ ]:
ho = np.dot(x, Wo) + bo  # (bs, 3), (3, 1), (1,) -> (bs, 1)
ho

In [ ]:
y_pred = sigmoid(ho)  # (bs, 1) -> (bs, 1)
y_pred

In [ ]:
MSE = np.square(y_true - y_pred)  # (bs, 1), (bs, 1) -> (bs, 1)
MSE

In [ ]:
y_pred_class = (y_pred > 0.5).astype(np.float)  # (bs, 1) -> (bs, 1)
y_pred_class

In [ ]:
y_match = (y_true == y_pred_class).astype(np.float)
y_match

In [ ]:
acc = np.sum(y_match) / max(y_true.shape[0], 1)  # (bs, 1) -> (1,)
acc

## backward

In [ ]:
dMSE_dy = -2 * (y_true - y_pred)  # (bs, 1), (bs, 1) -> (bs, 1)
dMSE_dy

In [ ]:
dy_dho = sigmoid_deriv(y_pred)  # (bs, 1) -> (bs, 1)
dy_dho  # (bs, 1)

In [ ]:
dMSE_dho = dMSE_dy * dy_dho  # (bs, 1), (bs, 1) -> (bs, 1)
dMSE_dho

In [ ]:
dho_dWo = x  # (bs, 3)
dho_dWo

In [ ]:
dho_dbo = np.array([[1, 1, 1, 1]]).T  # (bs, 1)
dho_dbo

In [ ]:
delta_Wo = np.dot(dho_dWo.T, dMSE_dho)  # (bs, 3).T, (bs, 1) -> (3, 1)
delta_Wo

In [ ]:
delta_bo = np.dot(dho_dbo.T, dMSE_dho)  # (bs, 1).T, (bs, 1) -> (1, 1)
delta_bo

In [ ]:
Wo -= lr * delta_Wo  # (3, 1), (3, 1) -> (3, 1)
Wo

In [ ]:
bo -= lr * delta_bo.squeeze()  # (1,), (1,) -> (1,)
bo

## train

In [ ]:
lr = 0.01

In [ ]:
Wo = np.random.randn(3, 1)  # (3, 1)
bo = np.zeros((1,))  # (1,)

Wo, bo

In [ ]:
losses, acces = [], []
for i in range(1000):
    # forward
    ho = np.dot(x, Wo) + bo  # (bs, 3), (3, 1), (1) -> (bs, 1)
    y_pred = sigmoid(ho)  # (bs, 1) -> (bs, 1)
    # loss
    MSE = np.square(y_true - y_pred)  # (bs, 1), (bs, 1) -> (bs, 1)
    losses.append(np.mean(MSE))  # (bs, 1) -> (1,)
    # acc
    y_match = (y_true == (y_pred > 0.5).astype(np.float)).astype(np.float)  # (bs, 1), (bs, 1) -> (bs, 1)
    acc = np.sum(y_match) / max(y_true.shape[0], 1)  # (bs, 1) -> 1
    acces.append(acc)

    # backward
    dMSE_dy = -2 * (y_true - y_pred)  # (bs, 1), (bs, 1) --> (bs, 1)
    dy_dho = sigmoid_deriv(y_pred)  # (bs, 1) -> (bs, 1)
    dMSE_dho = dMSE_dy * dy_dho  # (bs, 1) * (bs, 1) -> (bs, 1)
    dho_dWo = x  # (bs, 3)
    dho_dbo = np.array([[1, 1, 1, 1]]).T  # (bs, 1)
    # delta
    delta_Wo = np.dot(dho_dWo.T, dMSE_dho)  # (bs, 3).T, (bs, 1) -> (3, 1)
    delta_bo = np.dot(dho_dbo.T, dMSE_dho)  # (bs, 1).T, (bs, 1) -> (1, 1)
    # update weights
    Wo -= lr * delta_Wo
    bo -= lr * delta_bo.squeeze()

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(losses, 'b-', label='loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(acces, 'g-', label='acc')
plt.xlabel('Epoch')
plt.legend()

plt.show()